**Hi everyone! So today I'm going to be looking at inaccuracies in the NYT data for Alameda County that question the trustworthiness of their data collection. I am also going to compare this to the graphs the NYT has online when you search up Covid cases for Alameda County.**

I want to be clear that this is not a reflection of my political ideology and I'm not doing this to prove that COVID is a hoax or anything like that. I'm just doing this because I want people to do their own research on data before believing in something and because I want NYT to clear up the disrepancies in the data.

In [1]:
import numpy as np # linear algebra
import pandas as pd
import plotly.offline as py
py.init_notebook_mode(connected = True)
import plotly.graph_objs as go
from plotly.offline import plot, iplot
import plotly.express as px
pd.options.mode.chained_assignment = None

In [2]:
df = pd.read_csv('us-counties.csv')
df['date'] = pd.to_datetime(df['date'])
df.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0
3,2020-01-24,Cook,Illinois,17031.0,1,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0


There are multiple counties and states being used here but for our problem we're just going to focus on Alameda County in California.

In [3]:
alameda_county = df[(df['state'] == 'California') & (df['county'] == 'Alameda')]
px.bar(data_frame= alameda_county, x = 'date', y = 'cases', color = 'deaths')

Note how the data is always increasing. This is because the data is aggregated meaning that each case adds from the previous. The deaths are also aggregated. To unaggregate the data, we will use the .diff() function in pandas.

Strangely though, notice how towards the beginning of July 2021, the deaths all of the sudden go down and then start slowly increasing again. We will come back to this in a bit, for now lets see the results of unaggregating the data.

In [4]:
alameda_county['cases'] = alameda_county['cases'].diff()
alameda_county['deaths'] = alameda_county['deaths'].diff()
px.bar(data_frame= alameda_county, x = 'date', y = 'cases')

So for the most part, the number of cases when we take the difference seems fine, however you may notice that on July 3rd 2021 there seems to be -85 cases. Let's take a closer look at the data near the day.

In [5]:
px.bar(data_frame= alameda_county[(alameda_county['date'].dt.year == 2021) & (alameda_county['date'].dt.month == 7)], x = 'date', y = 'cases')

In [6]:
alameda_county[alameda_county['cases'] < 0]

,date,county,state,fips,cases,deaths
1478979,2021-07-03,Alameda,California,6001.0,-85.0,-1.0


In [7]:
df.iloc[[1478979]]

,date,county,state,fips,cases,deaths
1478979,2021-07-03,Alameda,California,6001.0,90396,1269.0


So for some reason July 3rd has -85 cases. On NYT's actual graph it says that on July 3rd there were 0 new cases in Alameda county. This is kind of worrying because it affects all the cases past this day and I know this is the same data.

Lets take a look at the deaths as well to see if their are any disrepancies there.

In [8]:
px.bar(data_frame=alameda_county, x = 'date', y = 'deaths')

Weirdly enough, according the the graph there is data that reaches about -400 deaths, making the result of the data look tiny in comparison. I'm going to graph this again but without that huge negative number so we can get a better idea of how the graph looks.

In [9]:
px.bar(data_frame=alameda_county[alameda_county['deaths'] > -400], x = 'date', y = 'deaths')

There still seems to be multiple points that are negative, let's check out all these points.

In [10]:
alameda_county = alameda_county.reset_index(drop = True)
alameda_county[alameda_county['deaths'] < 0]

,date,county,state,fips,cases,deaths
183,2020-08-31,Alameda,California,6001.0,147.0,-1.0
218,2020-10-05,Alameda,California,6001.0,38.0,-1.0
225,2020-10-12,Alameda,California,6001.0,67.0,-5.0
226,2020-10-13,Alameda,California,6001.0,63.0,-5.0
229,2020-10-16,Alameda,California,6001.0,38.0,-3.0
232,2020-10-19,Alameda,California,6001.0,102.0,-9.0
249,2020-11-05,Alameda,California,6001.0,137.0,-1.0
307,2021-01-02,Alameda,California,6001.0,1043.0,-1.0
320,2021-01-15,Alameda,California,6001.0,894.0,-8.0
321,2021-01-16,Alameda,California,6001.0,919.0,-11.0


Wow there are a lot of points on here just for Alameda County that have negative deaths. Let's compare it to what NYT actually puts out for the amount of deaths in some of these points.

In [11]:
alameda_county.iloc[[232]]

,date,county,state,fips,cases,deaths
232,2020-10-19,Alameda,California,6001.0,102.0,-9.0


On October 19th of 2020 we have -9 deaths.
 
NYT has the amount of deaths as 0.

In [12]:
alameda_county.iloc[[307]]

,date,county,state,fips,cases,deaths
307,2021-01-02,Alameda,California,6001.0,1043.0,-1.0


On January 2nd of 2021 we have -1 deaths

NYT has the amount of deaths as 0.

In [13]:
alameda_county.iloc[[384]]

,date,county,state,fips,cases,deaths
384,2021-03-20,Alameda,California,6001.0,80.0,-10.0


On March 20th of 2021 we have -10 deaths

NYT has the amount of deaths as 0.

In [14]:
alameda_county.iloc[[460]]

,date,county,state,fips,cases,deaths
460,2021-06-04,Alameda,California,6001.0,66.0,-423.0


On June 4th of 2021 we have -423 deaths

NYT has the amount of deaths as 0.

In [15]:
alameda_county.iloc[[492]]

,date,county,state,fips,cases,deaths
492,2021-07-06,Alameda,California,6001.0,108.0,-2.0


On July 6th of 2021 we have -2 deaths

NYT has the amount of deaths as 1.

I'm noticing a fairly common theme of putting 0 as the amount of deaths if the death counter is negative.

Again this is fairly worrying because this is real data that NYT uses and that millions of people see the graphs of everyday when looking at Covid cases and deaths, yet just in Alameda County we see multiple disrepancies in the data that really questions the validity of the data NYT has.

**Just to remind ourselves the magnitude of this problem, the data for deaths is cumulative in the original dataframe which means that everytime there is a new Covid death it adds up from the previous Covid deaths. Yet somehow for multiple dates, something went wrong in the cumulative process causing the amount of deaths to decrease from one day to another which is impossible. But if the deaths at some point decreased from one day to another, then how can we trust any of the data past the negative points because the next dates takes on the deaths from those disrepancies!**

Another thing that's worrying to note is how for July the amount of deaths is actually 1 as listed by the NYT instead of usually putting down 0 if the amount of deaths is negative. In fact the whole month of July is just weird, going back to our first cumulative graph.

In [16]:
alameda_county_2 = df[(df['state'] == 'California') & (df['county'] == 'Alameda')]
px.bar(data_frame= alameda_county_2, x = 'date', y = 'cases', color = 'deaths')

For the most part we can see that the amount of deaths are cumulative, indicated by the color getting lighter, and yet on June 4th, for some reason, the amount of deaths somehow decrease from 1687 to 1264 even though the data is cumulative. This trend for some reason then continues with the rest of the dates having somewhere near 1200 cumulative deaths and completely ignoring how it randomely decreased the amount of deaths.

Again there is no explanation offered for why there is a significantly huge change in the amount of deaths from June 3rd to June 4th and the amount of cases seems to still be cumulative (except on July 3rd where we see that the number of cases has randomely decreased and where our difference of -83 cases came from).

P.S. Just to note the change from June 3rd to June 4th is actually where the -423 deaths come in because we took the difference of June 4th - June 3rd or 1264 - 1687.

So at this point I'm very worried and doubtful about the trustworthiness of the NYT data. If for just Alameda County the amount of deaths and cases are this messed up, then I can't imagine how bad it may be for all the other counties out there.

**I've stated this once and I'll state this again, the data for deaths and cases are cumulative, so if there is any disrepancy for one day, it affects every other day after it, meaning that every day after it is wrong! So when the NYT posts data for the Alameda County and deaths somehow keep decreasing with cumulative data and no explanation, the data becomes untrustworthy**

I hope you've enjoyed reading this and now will go off to question data more often too instead of just believing a graph on a website. Have a great day!